In [1]:
spark

In [2]:
import json
import numpy as np
import time
from collections import Counter
import matplotlib
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
from operator import add

In [3]:
def preprocessing(data):
    print("data preprocessing...")
    data = data.filter((data.inflow_channel == "i") & (data.tabcode == "tot"))
    print("data preprocessing complete")
    return data

def make_df(path):
    print("data load...")
    return spark.read.json(path)

def make_count_dict(data):
    print("make dict...")
    return data.rdd.flatMap(lambda x : x["display"]["SER"]).map(lambda x : (x["code"],x["doc_count"]))\
        .reduceByKey(add).sortBy(lambda x : x[1],False)

In [4]:
mobile_data_path = '/Log/Type=Search/SubType1=Dialog/SubType2=Hour/Device=Mobile/Kind=Query/Year=2019/Month=07/Day=14'
pc_data_path = '/Log/Type=Search/SubType1=Dialog/SubType2=Hour/Device=PC/Kind=Query/Year=2019/Month=07/Day=14'
mobile_df = preprocessing(make_df(mobile_data_path))
pc_df = preprocessing(make_df(pc_data_path))

data load...
data preprocessing...
data preprocessing complete
data load...
data preprocessing...
data preprocessing complete


In [37]:
make_count_dict(mobile_df).take(10)

make dict...


[(u'NSC', 257628006),
 (u'IIM', 231135262),
 (u'TWB', 131242577),
 (u'TWA', 85202399),
 (u'CLO', 80052624),
 (u'NNS', 46313759),
 (u'VO2', 44833921),
 (u'SNP', 37185909),
 (u'IVR', 26315421),
 (u'0BL', 24764430)]

In [5]:
make_count_dict(pc_df).take(10)

make dict...


[(u'IIM', 91701663),
 (u'NNS', 39237224),
 (u'BR1', 28254358),
 (u'CCB', 26346517),
 (u'CLO', 25168798),
 (u'VO2', 23006597),
 (u'WSA', 22966032),
 (u'NKS', 18363604),
 (u'0PL', 14946573),
 (u'IVR', 14578113)]

In [20]:
def extract_unique_doc(data):
    print ("extracting ...")
    return data.rdd.flatMap(lambda x : x["display"]["SER"]).map(lambda x : (x["code"],x["documents"]))\
            .flatMapValues(lambda x : x).map(lambda x : ((x[0],x[1]["id"]),1)).reduceByKey(add)\
            .map(lambda x : (x[0][0],1)).reduceByKey(add).sortBy(lambda x : x[1], False)

In [22]:
extract_unique_doc(pc_df).collect()

extracting ...


[(u'IIM', 25025128),
 (u'BR1', 6342441),
 (u'WSA', 6213348),
 (u'CCB', 5203170),
 (u'NNS', 3603097),
 (u'VO2', 3359933),
 (u'SNP', 2782006),
 (u'NKS', 2138826),
 (u'IVR', 1273417),
 (u'GG2', 1211977),
 (u'KAS', 927689),
 (u'SNM', 799055),
 (u'W77', 603130),
 (u'KOQ', 506231),
 (u'1DV', 430298),
 (u'LB2', 354455),
 (u'BRC', 351052),
 (u'MS2', 288910),
 (u'0SC', 148427),
 (u'DIC', 135737),
 (u'0PL', 124644),
 (u'AYF', 114974),
 (u'0WD', 85965),
 (u'0BL', 75138),
 (u'RP1', 73741),
 (u'PRF', 59398),
 (u'0DL', 51175),
 (u'GG3', 41160),
 (u'L7T', 33247),
 (u'EM1', 30123),
 (u'P5T', 29592),
 (u'DIA', 29540),
 (u'GYB', 25951),
 (u'GGP', 25664),
 (u'TVP', 24117),
 (u'CRW', 21359),
 (u'LSU', 16586),
 (u'AEZ', 14450),
 (u'1SB', 11126),
 (u'GG1', 10817),
 (u'AQJ', 8818),
 (u'E4T', 7060),
 (u'SNE', 5619),
 (u'EMT', 4740),
 (u'Z8T', 4263),
 (u'VOL', 4263),
 (u'KDR', 2996),
 (u'CAR', 1697),
 (u'ATM', 1231),
 (u'BOC', 1174),
 (u'EA1', 1166),
 (u'CJK', 1094),
 (u'FES', 1083),
 (u'LAW', 1028),
 (u'CND',

In [23]:
extract_unique_doc(mobile_df).collect()

extracting ...


[(u'IIM', 48817154),
 (u'TWB', 21062832),
 (u'TWA', 17866430),
 (u'SNP', 5856999),
 (u'NSC', 4983270),
 (u'VO2', 4825388),
 (u'NNS', 3243302),
 (u'IVR', 1652466),
 (u'KAS', 1585108),
 (u'GG2', 1565205),
 (u'KOQ', 630206),
 (u'1DV', 580149),
 (u'MS2', 559507),
 (u'BRC', 427619),
 (u'AYF', 359826),
 (u'SNY', 349483),
 (u'LB2', 316186),
 (u'DIC', 192122),
 (u'PRF', 134301),
 (u'0BL', 120722),
 (u'0CL', 108935),
 (u'UUD', 108502),
 (u'GG3', 96359),
 (u'RP1', 92282),
 (u'L7T', 68596),
 (u'0EL', 64241),
 (u'0PL', 64208),
 (u'0DL', 62161),
 (u'LB0', 53550),
 (u'P5T', 45768),
 (u'CRW', 44503),
 (u'DIA', 44376),
 (u'GYB', 42704),
 (u'EM1', 38987),
 (u'GGP', 38164),
 (u'L1T', 37547),
 (u'TVP', 32573),
 (u'RP2', 32229),
 (u'BDS', 28484),
 (u'E4T', 23228),
 (u'AEZ', 19102),
 (u'LLB', 13265),
 (u'RP4', 9020),
 (u'SNQ', 8655),
 (u'VO3', 6219),
 (u'SNE', 5506),
 (u'3DV', 5319),
 (u'Z8T', 4661),
 (u'VOL', 4499),
 (u'AQJ', 4288),
 (u'RP5', 2946),
 (u'GZU', 2656),
 (u'1CI', 2186),
 (u'BOC', 1811),
 (u'E